<a href="https://colab.research.google.com/github/Alinaderi1378/Recomendation_film/blob/main/recommendation_film.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas scikit-learn nltk hazm

In [ ]:
# برای محیط دسکتاپ یا Google Colab
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import os

!pip install hazm
try:
    from hazm import stopwords_list, Normalizer
except ImportError:
    print("نصب hazm...")
    os.system("pip install hazm")
    from hazm import stopwords_list, Normalizer
# دانلود منابع لازم
nltk.download('punkt')

# خواندن داده‌ها
df = pd.read_csv('movies_farsi.csv', encoding='utf-8')
df['overview'] = df['overview'].fillna('')

In [ ]:


# پیش‌پردازش متن فارسی
normalizer = Normalizer()
stop_words = set(stopwords_list())

def preprocess(text):
    text = normalizer.normalize(text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in stop_words]
    return ' '.join(tokens)

df['overview_clean'] = df['overview'].apply(preprocess)

# بردارسازی با TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['overview_clean'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# تابع پیشنهاد فیلم
def get_recommendations(movie_title, top_n=3):
    if movie_title not in df['title'].values:
        return [f"فیلم «{movie_title}» یافت نشد."]
    idx = df[df['title'] == movie_title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    return [f"{df.iloc[i]['title']} (شباهت: {score:.2f})" for i, score in sim_scores]


print("🎬 لیست فیلم‌های موجود:")
for idx, title in enumerate(df['title']):
    print(f"{idx + 1}. {title}")

movie_index = int(input("\nشماره فیلم مورد نظر را وارد کنید: ")) - 1

if 0 <= movie_index < len(df):
    selected_title = df.iloc[movie_index]['title']
    print(f"\n✅ فیلم انتخاب‌شده: «{selected_title}»")
    print("🎯 پیشنهاد فیلم‌های مشابه:\n")
    recommendations = get_recommendations(selected_title)
    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec}")
else:
    print("شماره وارد شده نامعتبر است.")

